IMPORTS

In [1]:
import psycopg2
from Resources.config import PostgrePw
import pandas as pd

CONNECT TO LOCAL POSTGRESQL AND EXTRACT DATA

In [2]:
# Create the connection engine
conn_string = f"host='localhost' dbname='crime_data_db' user='postgres' password='{PostgrePw}'"
conn = psycopg2.connect(conn_string)

In [3]:
# establish a connection to Postgre
try:
    conn = psycopg2.connect(conn_string)
    print("Connection successful")
    
    # query and load the table
    query = "SELECT * FROM crime_data"
    
    # load into pandas DF
    crime_data_df = pd.read_sql_query(query,conn)
    
    # Display Df
    print(crime_data_df.head())
    
except Exception as e:
    print(f"Error connecting to the database: {e}")
# since the data is loaded into pandas, we can close the connection!
finally:
    if 'conn' in locals():
        conn.close()
        print("Connection closed")

Connection successful


C:\Users\Brittney Watts\AppData\Local\Temp\ipykernel_18712\2369206151.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  crime_data_df = pd.read_sql_query(query,conn)


              x              y  year incident_report_id             location  \
0  1.448107e+06  543688.000087  2021   20210807-1032-01         500 W 4TH ST   
1  1.406833e+06  499666.999875  2021   20210714-0714-02     14200 PERUGIA WY   
2  1.439801e+06  548658.999988  2017   20170521-1457-02         3000 RUSH AV   
3  1.449394e+06  572029.000163  2022   20221028-1713-02       4900 SUNSET RD   
4  1.432559e+06  550084.000090  2020   20201214-0857-00  4600 TUCKASEEGEE RD   

        city state    zip  x_coord_public  y_coord_public  ...  \
0  CHARLOTTE    NC  28202         1448107          543688  ...   
1  CHARLOTTE    NC  28273         1406833          499667  ...   
2  CHARLOTTE    NC  28208         1439801          548659  ...   
3  CHARLOTTE    NC  28269         1449394          572029  ...   
4  CHARLOTTE    NC   None         1432559          550084  ...   

   location_type_description  place_type_description  \
0                   Outdoors  Public/Non-Residential   
1         

In [4]:
# CHECK THE DATA TYPES, NUMBER OF RECORDS, AND COLUMN NAMES 
print(f'THE DATA TYPES:\n{crime_data_df.dtypes}\n')
print(f'NUMBER OF RECORDS: {len(crime_data_df)}\n')
print(f'COLUMN NAMES: {crime_data_df.columns}\n')
print(f'NUMBER OF UNIQUE VALUES PER COLUMNS:\n{crime_data_df.nunique()}')

THE DATA TYPES:
x                                   float64
y                                   float64
year                                  int64
incident_report_id                   object
location                             object
city                                 object
state                                object
zip                                  object
x_coord_public                        int64
y_coord_public                        int64
latitude_public                     float64
longitude_public                    float64
division_id                          object
cmpd_patrol_division                 object
npa                                   int64
date_reported                datetime64[ns]
date_incident_began                  object
date_incident_end                    object
address_description                  object
location_type_description            object
place_type_description               object
place_detail_description             object
clearance_status

CLEANING THE DATA EXTRACTED FROM POSTGRESQL

1. Cleaning clearance_status column to include open or cleared status

In [5]:
# checking cleared types 
clearance_types = crime_data_df['clearance_status'].value_counts()
print(clearance_types)

clearance_status
Open                                   394705
Exceptionally Cleared                  129424
Cleared by Arrest                      109573
Unfounded                               21443
Cleared by Arrest by Another Agency      4741
Name: count, dtype: int64


In [6]:
# Drop rows where clearance_status(target variable) is 'Unfounded'
clean_target_df = crime_data_df.loc[crime_data_df['clearance_status'] != 'Unfounded']

# Use .loc to map all statuses to 'Cleared' except for 'Open'
clean_target_df.loc[clean_target_df['clearance_status'] != 'Open', 'clearance_status'] = 'Cleared'
# show that rows have been deleted
print(len(clean_target_df))

638443


In [7]:
# Print the number of instances of different values in clearance status. there should only be two
clean_target_clearance_counts = clean_target_df['clearance_status'].value_counts()
print(clean_target_clearance_counts)

clearance_status
Open       394705
Cleared    243738
Name: count, dtype: int64


2. Ensure only Charlotte zipcodes are included in the DataFrame

In [8]:
# Filtering dataframe to include only charlotte zipcodes(there should oonly be 84 true clt zips)
# list of Charlotte ZIP codes
charlotte_zipcodes = [
    '28201', '28202', '28203', '28204', '28205', '28206', '28207', '28208', '28209', '28210',
    '28211', '28212', '28213', '28214', '28215', '28216', '28217', '28218', '28219', '28220',
    '28221', '28222', '28223', '28224', '28226', '28227', '28228', '28229', '28230', '28231',
    '28232', '28233', '28234', '28235', '28236', '28237', '28241', '28242', '28243', '28244',
    '28246', '28247', '28250', '28253', '28254', '28255', '28256', '28258', '28260', '28262',
    '28263', '28265', '28266', '28269', '28270', '28271', '28272', '28273', '28274', '28275',
    '28277', '28278', '28280', '28281', '28282', '28284', '28285', '28287', '28288', '28289',
    '28290', '28296', '28297', '28299'
]

# Filter DataFrame to only include rows with Charlotte ZIP codes
cleaned_zip_df = clean_target_df[clean_target_df['zip'].isin(charlotte_zipcodes)]
# verify that rows have been dropped
print(len(cleaned_zip_df))

427380


In [9]:
# Look at zip code value counts to identify and replace with "Other"
zip_type_value_counts = cleaned_zip_df['zip'].value_counts()

# Choose a cutoff value and create a list of zips to be dropped
zip_cutoff_value = 400
zips_to_drop = zip_type_value_counts[zip_type_value_counts < zip_cutoff_value].index

# Drop rows where zip codes are in the zips_to_drop list using .loc
cleaned_zip_df = cleaned_zip_df.loc[~cleaned_zip_df['zip'].isin(zips_to_drop)]


3. Reduce NPA value counts in the DataFrame

In [10]:
# Look at NPA value counts to identify and replace with "Other"
npa_value_counts = cleaned_zip_df['npa'].value_counts()
npa_value_counts

#npa_value_counts.to_list()

npa
476    21222
3       7479
122     7222
371     6267
331     5808
       ...  
453        1
463        1
404        1
438        1
426        1
Name: count, Length: 419, dtype: int64

In [11]:
cleaned_npa_df = cleaned_zip_df.copy()
# Choose a cutoff value and create a list of npa values to be dropped
npa_cutoff_value = 100
npas_to_drop = npa_value_counts[npa_value_counts < npa_cutoff_value].index
# Decrease the number of bins for optimization
npa_to_replace = npa_value_counts[npa_value_counts<100].index
# Drop rows where zip codes are in the zips_to_drop list using .loc
cleaned_npa_df = cleaned_npa_df.loc[~cleaned_npa_df['npa'].isin(npas_to_drop)]

# Check to make sure replacement was successful
cleaned_npa_df['npa'].value_counts()

npa
476    21222
3       7479
122     7222
371     6267
331     5808
       ...  
4        116
252      114
308      107
131      107
149      102
Name: count, Length: 377, dtype: int64

4. Change DATE_REPORTED date time to only the year and month so there are less columns to train. This could decrease the amount of columns from potentially 2,500+ to 84 for just dates. 

In [12]:
# see the value counts and length for date_reported since that is the next column that needs to be reduced
dates = cleaned_npa_df['date_reported'].value_counts()
dates

date_reported
2019-10-02    225
2017-09-18    225
2018-10-29    224
2019-05-20    224
2017-08-23    222
             ... 
2022-12-24     80
2018-12-25     77
2022-12-25     71
2022-11-24     70
2020-12-25     68
Name: count, Length: 2697, dtype: int64

In [13]:
cleaned_date_df = cleaned_npa_df.copy()
cleaned_date_df['date_reported'] = pd.to_datetime(cleaned_date_df['date_reported'])

# Change the datetime format to display only the year and month
cleaned_date_df['date_reported_y/m'] = cleaned_date_df['date_reported'].dt.strftime('%Y-%m')

# Display the value counts for date_reported_y/m to verify they have been changed
ym_dates = cleaned_date_df['date_reported_y/m'].value_counts()
ym_dates

date_reported_y/m
2019-10    5507
2017-05    5462
2017-08    5416
2018-08    5406
2019-07    5401
           ... 
2024-03    4249
2021-01    4180
2021-02    3966
2020-04    3806
2024-05    2898
Name: count, Length: 89, dtype: int64

In [14]:
# see what the datatype is for date_reported_y/m. it should now be object
cleaned_date_df.dtypes

x                                   float64
y                                   float64
year                                  int64
incident_report_id                   object
location                             object
city                                 object
state                                object
zip                                  object
x_coord_public                        int64
y_coord_public                        int64
latitude_public                     float64
longitude_public                    float64
division_id                          object
cmpd_patrol_division                 object
npa                                   int64
date_reported                datetime64[ns]
date_incident_began                  object
date_incident_end                    object
address_description                  object
location_type_description            object
place_type_description               object
place_detail_description             object
clearance_status                

5. Reduce value counts for 'place_detail_description'

In [15]:
# Look at place_detail_description value counts to identify and replace with "Other"
place_detail_description_value_counts = cleaned_date_df['place_detail_description'].value_counts()
place_detail_description_value_counts

place_detail_description
Private Residence                          106220
Apartment/Duplex Private Res                84263
Street/Highway                              41959
Other - Commercial Place                    24689
Hotel/Motel                                 13675
                                            ...  
Illegal/Unlicensed Liquor Establishment         6
Military Installation                           5
Mobile Home Dealership                          4
Bus - Non-Specific                              4
Taxi Company                                    1
Name: count, Length: 82, dtype: int64

In [16]:
cleaned_place_df = cleaned_date_df.copy()

# Choose a cutoff value and create a list of places to be dropped
place_cutoff_value = 100
places_to_drop = place_detail_description_value_counts[place_detail_description_value_counts < place_cutoff_value].index
# Decrease the number of bins for optimization
places_to_replace = place_detail_description_value_counts[place_detail_description_value_counts<100].index
# Drop rows where zip codes are in the zips_to_drop list using .loc
cleaned_place_df = cleaned_place_df.loc[~cleaned_place_df['place_detail_description'].isin(places_to_drop)]

# Check to make sure replacement was successful
cleaned_place_df['place_detail_description'].value_counts()

place_detail_description
Private Residence               106220
Apartment/Duplex Private Res     84263
Street/Highway                   41959
Other - Commercial Place         24689
Hotel/Motel                      13675
                                 ...  
Salvage Yard                       226
Community Center                   185
Jail/Prison                        147
Industrial Site                    128
Jewelry Store                      124
Name: count, Length: 63, dtype: int64

6. Reduce value counts for 'highest_nibrs_description'

In [17]:
# Look at highest_nibrs_code value counts to identify and replace with "Other"
highest_nibrs_description_value_counts = cleaned_place_df['highest_nibrs_description'].value_counts()
highest_nibrs_description_value_counts

highest_nibrs_description
Theft From Motor Vehicle       47643
All Other Thefts               42561
All Other Offenses             40305
Other Unlisted Non-Criminal    35084
Simple Assault                 31867
                               ...  
Betting/Wagering                   5
Welfare Fraud                      2
Bribery                            1
Purchasing Prostitution            1
Gas Leak                           1
Name: count, Length: 72, dtype: int64

In [18]:
cleanedish_nibrs_df = cleaned_date_df.copy()

# Choose a cutoff value and create a list of nibrs descriptions to be dropped
nibrs_cutoff_value = 100
nibrs_to_drop = highest_nibrs_description_value_counts[highest_nibrs_description_value_counts < nibrs_cutoff_value].index
# Decrease the number of bins for optimization
nibrs_to_replace = highest_nibrs_description_value_counts[highest_nibrs_description_value_counts<100].index
# Drop rows where zip codes are in the zips_to_drop list using .loc
cleanedish_nibrs_df = cleanedish_nibrs_df.loc[~cleanedish_nibrs_df['highest_nibrs_description'].isin(nibrs_to_drop)]

# Check to make sure replacement was successful
nibrs_counts = cleanedish_nibrs_df['highest_nibrs_description'].value_counts()
len(nibrs_counts)

54

NOW THAT ALL OF THE NECESSARY COLUMNS HAVE BEEN REDUCED, WE ARE NOW WORKING ON THE FINAL DATAFRAME

In [19]:
# verify the length of the dataframe
len(cleanedish_nibrs_df)

424863

In [20]:
# Because NIBRS codes in the 800 category are non-criminal offenses like missing person, suicide, and sudden death, we can remove rows where this is the case
# List of highest_nibrs_code values to remove
codes_to_remove = ['800', '801', '802', '803', '804', '806', '807', '809', '810', '899']

# Filter out rows where 'highest_nibrs_code' is in the list
cleaned_nibrs_df = cleanedish_nibrs_df[~cleanedish_nibrs_df['highest_nibrs_code'].isin(codes_to_remove)]
len(cleaned_nibrs_df)

367443

In [21]:
# drop columns that we don't need
crime_df = cleaned_nibrs_df[['year', 'zip', 'division_id', 'npa', 'date_reported_y/m', 
                   'place_detail_description', 'highest_nibrs_description', 'location_type_description', 'clearance_status']]
crime_df.head()

,year,zip,division_id,npa,date_reported_y/m,place_detail_description,highest_nibrs_description,location_type_description,clearance_status
0,2021,28202,01,476,2021-08,Air/Bus/Train Terminal,All Other Thefts,Outdoors,Open
1,2021,28273,21,82,2021-07,Apartment/Duplex Private Res,Theft From Motor Vehicle,Parking Lot,Open
2,2017,28208,02,293,2017-05,Private Residence,Damage/Vandalism Of Property,Outdoors,Open
3,2022,28269,11,125,2022-10,Hotel/Motel,Forcible Fondling,Indoors,Open
6,2017,28215,07,271,2017-08,Private Residence,Burglary/B&E,Indoors,Open


In [22]:
# Loop through the DataFrame to find and print null values' locations if there are any
for i, row in crime_df.iterrows():
    for column in crime_df.columns:
        if pd.isnull(row[column]):
            print(f"Null value found at row {i}, column '{column}'")

In [23]:
crime_df.to_csv('Resources/cleaned_crime_df.csv', index = False)

NO NULL VALUES WOOHOO. NOW WE CAN ENCODE THE DATA